# Preprocessing pipeline


This pipeline aims to serve as a semiautomatic and reproducible framework for preprocessing EEG signals before performing time-frequency-based analysis. It minimizes the manual steps required to clean the data based on visual inspection. It is advised to revisit the cleaned epochs before writing the final preprocessed file. 


## Outline

1. __Temporal filtering__

High-frequency artefacts and slow drifts are removed with a zero-phase bandpass filter using mne-Python [1]. The cutoff frequencies (0.5 - 45 Hz) can be modified in the utils folder in the configuration file (config.py). 


2. __Create epochs__ 

Epochs are nonoverlapping data segments created from the continuous data with a duration of 1 seconds. The length of epochs can be changed in the configuration file.
Epochs can be created from (1) events; there is a custom method that created epochs based on annotations in the raw data, (2) without events, data segments are created from the beginning of the raw data. 


3. __Outlier data rejection__  

    3.1. _Preliminar rejection_  
Epochs are rejected based on a global threshold on the z-score (> 3) of the epoch variance and amplitude range.

    3.2. _ICA decomposition_  
The default method is the infomax algorithm, however it can be changed in the configuration file along with the number of components and the decimation parameter. Components containing blink artefacts are automatically marked with mne-Python.
The ICA sourced can be visualized and interactively selected and rejected based on their topographies, time-courses or frequency spectra. The number of components that were removed from the data are documented in the “description” field of the epochs instance “info” structure.

    3.3. _Autoreject_  
Autoreject [2, 3] uses unsupervised learning to estimate the rejection threshold for the epochs. In order to reduce computation time that increases with the number of segments and channels, autoreject can be fitted on a representative subset of epochs (25% of total epochs). Once the parameters are learned, the solution can be applied to any data that contains channels that were used during fit.


4. __Outlier channel interpolation__

The Random Sample Consensus (RANSAC) algorithm [4] selects a random subsample of good channels to make predictions of each channel in small non-overlapping 4 seconds long time windows. It uses a method of spherical splines (Perrin et al., 1989) to interpolate the bad sensors. The sensors that were interpolated are added to the "description" field of the epochs "info" structure. 


<img src="static/preprocessing_pipeline_diagram.svg">


## References

[1] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013, ISSN 1662-453X

[2] Mainak Jas, Denis Engemann, Federico Raimondo, Yousra Bekhti, and Alexandre Gramfort, “Automated rejection and repair of bad trials in MEG/EEG.” In 6th International Workshop on Pattern Recognition in Neuroimaging (PRNI), 2016.

[3] Mainak Jas, Denis Engemann, Yousra Bekhti, Federico Raimondo, and Alexandre Gramfort. 2017. “Autoreject: Automated artifact rejection for MEG and EEG data”. NeuroImage, 159, 417-429.

[4] Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., & Robbins, K. A. (2015). The PREP pipeline: standardized preprocessing for large-scale EEG analysis. Frontiers in neuroinformatics, 9, 16.



## Import packages


```%matplotlib qt``` is the recommended backend for interactive visualization (can be slower);    

switch to ```%matplotlib inline``` for (faster) static plots

In [19]:
import os
from ipyfilechooser import FileChooser

from meeg_tools.preprocessing import *
from meeg_tools.utils.epochs import create_epochs_from_events, create_metadata
from meeg_tools.utils.raw import read_raw_measurement, filter_raw
from meeg_tools.utils.log import update_log

from meeg_tools.utils.config import settings

from matplotlib import pyplot as plt
%matplotlib qt

## Load raw data


See [this](https://mne.tools/stable/auto_tutorials/io/20_reading_eeg_data.html) documentation for help with supported file formats.  

In [2]:
# Use the widget to navigate to the experiment folder path and select an EEG file 
base_path = 'D:/TMS_rewiring/'
fc = FileChooser(base_path)
fc.filter_pattern = ['*.vhdr', '*.edf']

display(fc)

FileChooser(path='D:\TMS_rewiring', filename='', title='', show_hidden=False, select_desc='Select', change_des…

In [3]:
# Load selected file
raw = read_raw_measurement(raw_file_path=fc.selected)
raw.info

Extracting parameters from D:\TMS_rewiring\Raw_data\17_E\Day1\EEG\17_E_Day1.vhdr...
Setting channel info structure...


<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 64 EEG
 custom_ref_applied: False
 dig: 64 items (64 EEG)
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2020-12-17 09:26:36 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 temp: 17_E_Day1
>

## Temporal filtering

- bandpass filter (0.5 - 45 Hz)

In [4]:
raw_bandpass = filter_raw(raw)

Reading 0 ... 3294249  =      0.000 ...  6588.498 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 3301 samples (6.602 sec)



[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done  64 out of  64 | elapsed:    8.0s finished


## Create epochs

- select the events for analysis

In [5]:
settings['epochs']['start_time'] = -0.25
settings['epochs']['end_time'] = 0.75

In [6]:
events_ids = np.concatenate([np.arange(10, 53, 1), 
                             np.arange(10, 53, 1) + 100,
                            [211, 212, 213, 214, 215, 216]])

epochs = create_epochs_from_events(raw=raw_bandpass, event_ids=[events_ids])

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  5', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 22', 'Stimulus/S 24', 'Stimulus/S 25', 'Stimulus/S 26', 'Stimulus/S 27', 'Stimulus/S 28', 'Stimulus/S 29', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 32', 'Stimulus/S 34', 'Stimulus/S 35', 'Stimulus/S 36', 'Stimulus/S 37', 'Stimulus/S 38', 'Stimulus/S 39', 'Stimulus/S 40', 'Stimulus/S 41', 'Stimulus/S 42', 'Stimulus/S 43', 'Stimulus/S 44', 'Stimulus/S 45', 'Stimulus/S 46', 'Stimulus/S 47', 'Stimulus/S 48', 'Stimulus/S 49', 'Stimulus/S 51', 'Stimulus/S 52', 'Stimulus/S 61', 'Stimulus/S 62', 'Stimulus/S 63', 'Stimulus/S 64', 'Stimulus/S 65', 'Stimulus/S 66', 'Stimulus/S 67', 'Stimulus/S 68', 'Stimulus/S 69', 'Stimulus/S 70', 'Stimulus/S 71', 'Stimulus/S 72', 'Stimulus/S 75', 'Stimulus/S 76', 'Stimulus/S 77', 'Stimulus/S 78', 

## Create metadata for epochs (optional)

- adding metadata makes it easier to select epochs of different types
- custom triggers are selected from the raw instance

- metadata can be added or replaced later (e.g. after preprocessing)

In [7]:
metadata = create_metadata(epochs)
metadata.head(10)

epochs.metadata = metadata

Found these indices for these epoch boundary events: 
211	1311
212	2615
213	3936
214	5247
215	6570
Adding metadata with 8 columns


In [8]:
# subselecting epochs 
# Here we could also include thrills, repetitions, or practice stimuli.
# ICA should not run on duplicate data (epochs should not be overlapping!)

epochs = epochs["triplet == 'L' | triplet == 'H'"]
epochs = epochs["answer == 'correct'"]

## Run preprocessing


### 1.1. Preliminary epoch rejection

In [9]:
epochs_faster = prepare_epochs_for_ica(epochs=epochs)

Preliminary epoch rejection: 
Loading data for 1630 events and 501 original time points ...
0 bad epochs dropped
Bad epochs by amplitude
	[ 115  468  532  533  583  622  712  924  933  966 1066 1078 1092 1103
 1178 1228 1243 1258 1277 1284 1312 1336 1337 1338 1352 1353 1354 1357
 1366 1378 1380 1383 1404 1416 1421 1446 1456 1459 1463 1464 1474 1478
 1490 1504 1513 1516 1520 1522 1524 1526 1528 1533 1544 1548 1553 1587
 1602 1603 1609 1611 1620]
Bad epochs by deviation
	[ 491  583  596  652  790  814  915 1042 1149 1277 1337 1352 1354 1378
 1382 1383 1398 1406 1416 1446 1490 1522 1528 1533 1543 1553]
Bad epochs by variance
	[ 468  532  533  583  589  622  651  657  666  712  718  915  924  947
  966 1078 1092 1103 1117 1168 1178 1212 1217 1228 1229 1243 1258 1277
 1284 1336 1337 1351 1352 1353 1354 1356 1357 1366 1378 1380 1382 1383
 1384 1416 1436 1453 1456 1459 1463 1464 1474 1478 1485 1487 1490 1496
 1501 1512 1513 1516 1520 1522 1524 1526 1528 1529 1530 1533 1538 1542
 1543 1544 154

### 1.2. Run ICA


The parameters are: 32 ICA components using ["infomax"](https://mne.tools/stable/generated/mne.preprocessing.infomax.html) algorithm. 

When visualizing the components, it is recommended to subset the data (see below).

In [10]:
settings['ica']['n_components'] = 32
settings['ica']['method'] = 'picard'

In [11]:
ica = run_ica(epochs=epochs_faster, fit_params=dict(ortho=True))

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Loading data for 1524 events and 501 original time points ...
Selecting by number: 32 components
Loading data for 1524 events and 501 original time points ...
Fitting ICA took 66.3s.
EOG channels are not found. Attempting to use Fp1,Fp2 channels as EOG channels.
Using EOG channels: Fp1, Fp2
Loading data for 1524 events and 501 original time points ...
Loading data for 1524 events and 501 original time points ...
Loading data for 1524 events and 501 original time points ...
Loading data for 1524 events and 501 original time points ...


In [ ]:
# Plot component topographies
ica.plot_components()

In [ ]:
# Visualize components on epochs
# Subset epochs to reduce execution time (e.g. take epochs from every 7th event)
subset = list(epochs.event_id.keys())[::7]
# Exclude components by selecting them, right click on component name to visulize source:
ica.plot_sources(epochs_faster[subset])

In [12]:
# After selecting the components to exclude, apply ICA to epochs
# Documents the number of excluded components
epochs_ica = apply_ica(epochs=epochs_faster, ica=ica)

Loading data for 1524 events and 501 original time points ...
Applying ICA to Epochs instance
    Transforming to ICA space (32 components)
    Zeroing out 1 ICA component
    Projecting back using 64 PCA components


In [13]:
epochs_ica.info['description']

'n_components: 1'

### 1.3. Visualize ICA cleaned epochs (optional)

This step can be repeated after each preprocessing step, or you can also do a final inspection at the end. 

In [ ]:
# Optional
epochs_ica[subset].plot(n_epochs=10, n_channels=32, scalings={'eeg': 20e-6},)

### 1.4. Save cleaned epochs (recommended)

In [15]:
# Create folder for preprocessed and interim files
folder_name = 'preprocessed'
epochs_path = os.path.join(base_path, folder_name, 'epochs_asrt')


# Create path to epoch files
if not os.path.exists(epochs_path):
    os.makedirs(epochs_path)

# Save ICA cleaned epochs 
postfix = '-epo.fif.gz'
epochs_ica.save(os.path.join(epochs_path, f'{epochs_ica.info["temp"]}{postfix}'), overwrite=True)

In [ ]:
print(epochs_ica.info)

### 1.5. Create a log file 

We can create a log file for the preprocessed data and store metadata
that could be useful to remember. You can add more columns to this, or 
remove the ones that are not needed. For documentation purporses, it is 
recommended to store the number of rejected and total epochs, the number of
ICA components that were rejected, the number of interpolated electrodes etc.
You can also add a column with "notes" to add custom descriptions about the data.

In [16]:
# Specify path to log file 
log_file_path = os.path.join(epochs_path, 'log.csv')
# Add description for the preprocessed data (optional)
notes = ''

In [17]:
update_log(log_file_path, epochs_ica, notes)

,fid,highpass,lowpass,n_components,n_bad_epochs,n_total_epochs,drop_percentage,stimuli,t_min,t_max,n_interpolated,average_ref_applied,baseline,notes,date_of_update
0,17_E_Day1_ICA,0.5,45.0,1.0,106,1524,6.5,"[10, 11, 12, 14, 15, 16]",-0.25,0.75,NaN,False,NaN,,2021-11-25T15:37:29.650122


### 2.1. Run autoreject

In [18]:
reject_log = run_autoreject(epochs_ica, n_jobs=11, subset=True)

Fitting autoreject on random (n=381) subset of epochs: 
Running autoreject on ch_type=eeg


Computing thresholds ...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:24<00:00,  2.64it/s]


Repairing epochs:   0%|                                                                                                                     | 0/381 [00:00<?, ?it/s]

Repairing epochs:   4%|████▍                                                                                                      | 16/381 [00:00<00:02, 159.30it/s]

Repairing epochs:  11%|███████████▌                                                                                               | 41/381 [00:00<00:01, 211.14it/s]

Repairing epochs:  18%|██████████████████▊                                                                                        | 67/381 [00:00<00:01, 232.32it/s]

Repairing epochs:  24%|██████████████████████████                                                                                 | 93/381 [00:00<00:01, 241.27it/s]

Rep

Repairing epochs:  51%|██████████████████████████████████████████████████████▊                                                    | 195/381 [00:03<00:03, 48.14it/s]

Repairing epochs:  53%|████████████████████████████████████████████████████████▍                                                  | 201/381 [00:03<00:03, 49.31it/s]

Repairing epochs:  54%|██████████████████████████████████████████████████████████▏                                                | 207/381 [00:03<00:03, 51.02it/s]

Repairing epochs:  56%|███████████████████████████████████████████████████████████▊                                               | 213/381 [00:03<00:03, 50.77it/s]

Repairing epochs:  57%|█████████████████████████████████████████████████████████████▌                                             | 219/381 [00:04<00:03, 49.79it/s]

Repairing epochs:  60%|████████████████████████████████████████████████████████████████                                           | 228/381 [00:04<00:02, 58.16it/s]

Repa

Repairing epochs:  13%|█████████████▉                                                                                              | 49/381 [00:00<00:05, 64.22it/s]

Repairing epochs:  15%|███████████████▊                                                                                            | 56/381 [00:00<00:05, 61.61it/s]

Repairing epochs:  17%|██████████████████▏                                                                                         | 64/381 [00:00<00:04, 66.33it/s]

Repairing epochs:  19%|████████████████████▋                                                                                       | 73/381 [00:01<00:04, 70.14it/s]

Repairing epochs:  21%|██████████████████████▉                                                                                     | 81/381 [00:01<00:04, 69.68it/s]

Repairing epochs:  23%|█████████████████████████▏                                                                                  | 89/381 [00:01<00:04, 65.44it/s]

Repa

Fold:  10%|████████████▏                                                                                                             | 1/10 [00:01<00:14,  1.59s/it]


Fold:  20%|████████████████████████▍                                                                                                 | 2/10 [00:03<00:12,  1.59s/it]


Fold:  30%|████████████████████████████████████▌                                                                                     | 3/10 [00:04<00:11,  1.58s/it]


Fold:  40%|████████████████████████████████████████████████▊                                                                         | 4/10 [00:06<00:09,  1.58s/it]


Fold:  50%|█████████████████████████████████████████████████████████████                                                             | 5/10 [00:07<00:07,  1.59s/it]


Fold:  60%|█████████████████████████████████████████████████████████████████████████▏                                                | 6/10 [00:09<00:06,  1.58s/it]


Repairing epochs:  72%|████████████████████████████████████████████████████████████████████████████▋                              | 273/381 [00:04<00:01, 62.37it/s]

Repairing epochs:  73%|██████████████████████████████████████████████████████████████████████████████▋                            | 280/381 [00:04<00:01, 55.23it/s]

Repairing epochs:  75%|████████████████████████████████████████████████████████████████████████████████▌                          | 287/381 [00:04<00:01, 55.18it/s]

Repairing epochs:  77%|██████████████████████████████████████████████████████████████████████████████████▌                        | 294/381 [00:04<00:01, 56.74it/s]

Repairing epochs:  79%|████████████████████████████████████████████████████████████████████████████████████▎                      | 300/381 [00:04<00:01, 54.22it/s]

Repairing epochs:  80%|█████████████████████████████████████████████████████████████████████████████████████▉                     | 306/381 [00:05<00:01, 55.31it/s]

Repa





Estimated consensus=0.30 and n_interpolate=4

AUTOREJECT report
There are 32 bad epochs found with Autoreject. You can assess these epochs with reject_log.bad_epochs

There are 88 bad epochs where more than 15% of the channels were noisy. You can assess these epochs with reject_log.report


In [20]:
# Here you can decide how strict should be the epoch rejection.
# You can drop only those that were marked as bad epochs, or a more 
# strict rejection threshold can be if you drop epochs where more than
# 15% of the channels were marked as noisy.

# You can plot the epochs with Autoreject, where bad epochs are marked with
# red colors. 

# reject_log.plot_epochs(epochs_faster)


# rejecting only bad epochs
# epochs_autoreject = epochs_faster.copy().drop(reject_log.bad_epochs, reason='AUTOREJECT')

# rejecting those epochs too where more than 15% of the channels are marked as noisy
#bads = np.where(np.count_nonzero(reject_log.labels, axis=1) > 0.15 * epochs_faster.info['nchan'])[0].tolist()

# you can plot just the bad epochs to double check how strict this rejection is
# if bads: 
#     epochs_faster[bads].plot(n_epochs=10,
#                                 scalings={'eeg': 20e-6},
#                                 n_channels=32)


epochs_autoreject = epochs_ica.copy().drop(reject_log.report, reason='AUTOREJECT')

Dropped 88 epochs: 8, 27, 33, 65, 165, 190, 206, 225, 277, 278, 280, 293, 317, 320, 328, 331, 333, 398, 401, 452, 463, 476, 495, 505, 507, 516, 517, 518, 558, 566, 588, 608, 625, 642, 659, 690, 712, 714, 720, 730, 761, 770, 775, 783, 795, 797, 848, 864, 871, 876, 898, 901, 908, 937, 978, 993, 994, 1023, 1024, 1035, 1060, 1120, 1121, 1126, 1147, 1148, 1154, 1174, 1175, 1182, 1222, 1239, 1245, 1248, 1273, 1300, 1342, 1358, 1359, 1360, 1375, 1384, 1391, 1413, 1445, 1448, 1454, 1468


In [22]:
# save clean epochs
fid = epochs_ica.info['temp']
epochs_autoreject.info.update(temp=f'{fid}_autoreject')

print(epochs_autoreject.info['temp'])
postfix = '-epo.fif.gz'
epochs_autoreject.save(os.path.join(epochs_path, f'{epochs_autoreject.info["temp"]}{postfix}'), overwrite=True)

17_E_Day1_ICA_autoreject


In [23]:
epochs_autoreject.info['temp']

'17_E_Day1_ICA_autoreject'

In [24]:
update_log(log_file_path, epochs_autoreject, '')

,fid,highpass,lowpass,n_components,n_bad_epochs,n_total_epochs,drop_percentage,stimuli,t_min,t_max,n_interpolated,average_ref_applied,baseline,notes,date_of_update
0,17_E_Day1_ICA_autoreject,0.5,45.0,1.0,194,1436,11.9,"[10, 11, 12, 14, 15, 16]",-0.25,0.75,NaN,False,NaN,,2021-11-25T15:40:10.273933


In [25]:
epochs_autoreject.info['description']

'n_components: 1'

### 3. Run ransac

In [26]:
bads = get_noisy_channels(epochs=epochs_autoreject, with_ransac=True)

Creating RawArray with float64 data, n_channels=64, n_times=719436
    Range : 0 ... 719435 =      0.000 ...  1438.870 secs
Ready.
Executing RANSAC
This may take a while, so be patient...
Progress: 10%... 20%... 30%... 40%... 50%... 60%... 70%... 80%... 90%... 100%

RANSAC done!

NoisyChannels REPORT
------------------------
8.0% of the channels were detected as noisy.
(5) channels: P3, AF7, Fp2, Fp1, CP5


In [27]:
epochs_ransac = interpolate_bad_channels(epochs=epochs_autoreject, bads=bads)

Interpolating bad channels
    Automatic origin fit: head of radius 95.0 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 5 sensors


In [28]:
# Check how many trials are left for each condition per epoch
for i in range(5):
    print(i+1, epochs_ransac[f"epoch == {i+1}& triplet == 'L'"].average().nave)

1 93
2 92
3 78
4 76
5 65


In [29]:
# inspect which sensors were interpolated (if any)
print(epochs_ransac.info)

<Info | 10 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 64 EEG
 custom_ref_applied: False
 description: n_components: 1, interpolated: P3, AF7, Fp2, Fp1, CP5
 dig: 64 items (64 EEG)
 highpass: 0.5 Hz
 lowpass: 45.0 Hz
 meas_date: 2020-12-17 09:26:36 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 temp: 17_E_Day1_ICA_autoreject
>


## 4. Final visual inspection

Mark epochs that should be dropped,  etc.

In [ ]:
# use indexing to plot fewer epochs (faster) e.g. [::7] shows only every 7th epoch
epochs_ransac[::7].plot(n_epochs=10,
                       n_channels=32,
                # group_by='position',
                       scalings={'eeg': 20e-6})

### 5.2. Set average reference

To set a “virtual reference” that is the average of all channels, you can use set_eeg_reference() with ref_channels='average'.

In [30]:
epochs_ransac.set_eeg_reference('average')

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Number of events,1436
Events,10: 15311: 62012: 15114: 5015: 20916: 253
Time range,-0.250 – 0.750 sec
Baseline,off


## 6. Save cleaned epochs

In [31]:
epochs_ransac.plot_drop_log()

# save clean epochs
fid = epochs_autoreject.info['temp']
epochs_ransac.info.update(temp=f'{fid}_ransac')

print(epochs_ransac.info["temp"])
postfix = '-epo.fif.gz'
epochs_ransac.save(os.path.join(epochs_path, f'{epochs_ransac.info["temp"]}{postfix}'), overwrite=True)

17_E_Day1_ICA_autoreject_ransac


In [32]:
notes = ''
update_log(log_file_path, epochs_ransac, notes)

,fid,highpass,lowpass,n_components,n_bad_epochs,n_total_epochs,drop_percentage,stimuli,t_min,t_max,n_interpolated,average_ref_applied,baseline,notes,date_of_update
0,17_E_Day1_ICA_autoreject_ransac,0.5,45.0,1.0,194,1436,11.9,"[10, 11, 12, 14, 15, 16]",-0.25,0.75,5.0,True,NaN,,2021-11-25T15:41:48.822459


## Time-frequency analysis
### Evoked


In [ ]:
# Subset channels

ch_names = ['F7', 'F5', 'F3', 'FC5', 'FC3',
           'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2',
           'F4', 'F6', 'F8', 'FC4', 'FC6',
           'FT7', 'T7', 'TP7', 
           'C3', 'Cz', 'C4',
           'FT8', 'T8', 'TP8',
           'CP5', 'CP3', 'P7', 'P5', 'P3',
           'CP1', 'CPz', 'CP2', 'P1', 'Pz', 'P2',
           'CP4', 'CP6', 'P4', 'P6', 'P8',
           'PO3', 'PO7', 'O1',
           'PO4', 'PO8', 'O2',]

epochs_evoked = epochs_ransac.copy().pick_channels(ch_names, ordered=True)

In [ ]:
epochs_evoked.info

In [ ]:
epochs_evoked.comment = epochs_ransac.info['fid'].split('_')[0]
    
e1_H = epochs_evoked["epoch == 1 & triplet == 'H'"].average()
e1_H.apply_baseline((-0.2, 0.0))
e1_H.comment = f"{epochs_evoked.comment}_e1_H"
    
e1_L = epochs_evoked["epoch == 1 & triplet == 'L'"].average()
e1_L.apply_baseline((-0.2, 0.0))
e1_L.comment = f"{epochs_evoked.comment}_e1_L"
    
e2_H = epochs_evoked["epoch == 2 & triplet == 'H'"].average()
e2_H.apply_baseline((-0.2, 0.0))
e2_H.comment = f"{epochs_evoked.comment}_e2_H"
    
e2_L = epochs_evoked["epoch == 2 & triplet == 'L'"].average()
e2_L.apply_baseline((-0.2, 0.0))
e2_L.comment = f"{epochs_evoked.comment}_e2_L"
    
e3_H = epochs_evoked["epoch == 3 & triplet == 'H'"].average()
e3_H.apply_baseline((-0.2, 0.0))
e3_H.comment = f"{epochs_evoked.comment}_e3_H"
    
e3_L = epochs_evoked["epoch == 3 & triplet == 'L'"].average()
e3_L.apply_baseline((-0.2, 0.0))
e3_L.comment = f"{epochs_evoked.comment}_e3_L"
    
e4_H = epochs_evoked["epoch == 4 & triplet == 'H'"].average()
e4_H.apply_baseline((-0.2, 0.0))
e4_H.comment = f"{epochs_evoked.comment}_e4_H"
    
e4_L = epochs_evoked["epoch == 4 & triplet == 'L'"].average()
e4_L.apply_baseline((-0.2, 0.0))
e4_L.comment = f"{epochs_evoked.comment}_e4_L"
    
e5_H = epochs_evoked["epoch == 5 & triplet == 'H'"].average()
e5_H.apply_baseline((-0.2, 0.0))
e5_H.comment = f"{epochs_evoked.comment}_e5_H"
    
e5_L = epochs_evoked["epoch == 5 & triplet == 'L'"].average()
e5_L.apply_baseline((-0.2, 0.0))
e5_L.comment = f"{epochs_evoked.comment}_e5_L"
    
fig, ax = plt.subplots(3, 5, figsize=(11.69,8.27), sharex=True)
fig.suptitle(epochs_evoked.info['description'])
mne.viz.plot_compare_evokeds([e1_H, e1_L], combine='mean', axes=ax[0,0], show=False)
mne.viz.plot_compare_evokeds([e2_H, e2_L], combine='mean', axes=ax[0,1], show=False)
mne.viz.plot_compare_evokeds([e3_H, e3_L], combine='mean', axes=ax[0,2], show=False)
mne.viz.plot_compare_evokeds([e4_H, e4_L], combine='mean', axes=ax[0,3], show=False)
mne.viz.plot_compare_evokeds([e5_H, e5_L], combine='mean', axes=ax[0,4], show=False)
e1_H.plot(spatial_colors=True, axes=ax[1,0], window_title='', show=False)
e1_L.plot(spatial_colors=True, axes=ax[2,0], window_title='', show=False)
e2_H.plot(spatial_colors=True, axes=ax[1,1], window_title='', show=False)
e2_L.plot(spatial_colors=True, axes=ax[2,1], window_title='', show=False)
e3_H.plot(spatial_colors=True, axes=ax[1,2], window_title='', show=False)
e3_L.plot(spatial_colors=True, axes=ax[2,2], window_title='', show=False)
e4_H.plot(spatial_colors=True, axes=ax[1,3], window_title='', show=False)
e4_L.plot(spatial_colors=True, axes=ax[2,3], window_title='', show=False)
e5_H.plot(spatial_colors=True, axes=ax[1,4], window_title='', show=False)
e5_L.plot(spatial_colors=True, axes=ax[2,4], window_title='', show=False)
ax[1,0].set_title('')
ax[1,1].set_title('')
ax[1,2].set_title('')
ax[1,3].set_title('')
ax[1,4].set_title('')
ax[2,0].set_title('')
ax[2,1].set_title('')
ax[2,2].set_title('')
ax[2,3].set_title('')
ax[2,4].set_title('')
fig.savefig(os.path.join(epochs_path, f'{epochs_evoked.comment}_E_evoked.pdf', ),dpi=200)
fig.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11.69,8.27), sharex=True)
e1_H.plot_topo(axes=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11.69,8.27), sharex=True)
e1_L.plot_topo(axes=ax)

In [ ]:
epochs_evoked["epoch == 1 & triplet == 'H'"].plot_psd()